# 1. Unzip Data

In [74]:
import zipfile
zipfile.ZipFile('data/archive.zip', 'r').extractall('data/')

# 2. Read Datasets
   

In [75]:
import numpy as np
import struct

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromfile(f, dtype=np.uint8).reshape(shape)
    

X_train = read_idx('data/train-images-idx3-ubyte/train-images-idx3-ubyte')
y_train = read_idx('data/train-labels-idx1-ubyte/train-labels-idx1-ubyte')
X_test = read_idx('data/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
y_test = read_idx('data/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


# 3. Convert data to TensorDatasets

In [76]:
from fastai.vision.all import *
import torch

# Konvertieren Sie die Numpy-Arrays in TensorDatasets
train_ds = torch.utils.data.TensorDataset(torch.tensor(X_train).float().view(-1, 1, 28, 28).repeat(1, 3, 1, 1), torch.tensor(y_train)) # repeat(1, 3, 1, 1) um die Anzahl der Kanäle auf 3 zu erhöhen
valid_ds = torch.utils.data.TensorDataset(torch.tensor(X_test).float().view(-1, 1, 28, 28).repeat(1, 3, 1, 1), torch.tensor(y_test)) # repeat(1, 3, 1, 1) um die Anzahl der Kanäle auf 3 zu erhöhen

train_ds.tensors[0].shape, train_ds.tensors[1].shape, valid_ds.tensors[0].shape, valid_ds.tensors[1].shape

(torch.Size([60000, 3, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 3, 28, 28]),
 torch.Size([10000]))

In [77]:
# Erstellen Sie DataLoaders aus den Datasets über fastai
dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=64, shuffle=True, num_workers=4)
dls.tensors[0].shape, dls.tensors[1].shape, dls.valid.tensors[0].shape, dls.valid.tensors[1].shape

(torch.Size([60000, 3, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 3, 28, 28]),
 torch.Size([10000]))

# 4. Learning

In [78]:
from fastai.vision.all import *
from fastai.losses import CrossEntropyLossFlat


simple_net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28*3,50),
    nn.ReLU(),
    nn.Linear(50,30),
    nn.ReLU(),
    nn.Linear(30,20),
    nn.ReLU(),
    nn.Linear(20,30),
    nn.ReLU(),
    nn.Linear(30,10)
)

learn = Learner(dls, simple_net, metrics=accuracy, loss_func=CrossEntropyLossFlat())

# Trainieren Sie das Modell
learn.fit_one_cycle(10, 0.01)

epoch,train_loss,valid_loss,accuracy,time
0,0.345337,0.324343,0.905300,00:04
1,0.415695,0.400312,0.905600,00:04
2,0.330152,0.317424,0.926000,00:04
3,0.315572,0.255541,0.942700,00:04
4,0.269678,0.235846,0.944700,00:04
5,0.215857,0.233708,0.943400,00:04
6,0.177657,0.194657,0.956100,00:04
7,0.123832,0.156872,0.962700,00:04
8,0.123896,0.147977,0.962900,00:04
9,0.097362,0.148248,0.964600,00:04


In [79]:
learn = vision_learner(dls, resnet18, pretrained=False,
                    loss_func=F.cross_entropy, metrics=accuracy, n_out=10)
learn.fit_one_cycle(1, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.095909,0.052787,0.987300,02:03
